In [1]:
import pandas as pd
import csv
import os
import json
import numpy as np
from citipy import citipy
import requests
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
results_df = pd.read_csv("../WeatherPy/results.csv")
results_df

,Unnamed: 0,City,ID,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness
0,0,Albany,5106841,42.6001,-73.9662,37.99,77,5.46,100
1,1,Bluff,2206939,-46.6000,168.3333,63.01,79,15.84,13
2,2,Gardabani,614575,41.4605,45.0928,39.20,81,14.97,40
3,3,Cidreira,3466165,-30.1811,-50.2056,76.05,80,20.78,86
4,4,Castro,3466704,-24.7911,-50.0119,61.77,98,5.35,35
...,...,...,...,...,...,...,...,...,...
676,676,Ransang,1691872,8.9603,117.5828,82.44,78,5.70,100
677,677,Artvin Province,751816,41.1667,41.8333,41.00,87,6.91,75
678,678,Calvinia,3369174,-31.4707,19.7760,56.39,70,6.80,4
679,679,Coahuayana Viejo,4013679,18.7333,-103.6833,70.88,57,1.57,100


In [3]:
results_df_filtered = results_df[(results_df["Max Temp"] <80) & (results_df["Max Temp"] >70) & (results_df["Wind Speed"] <10) & (results_df["Cloudiness"] == 0)]
results_df_filtered

,Unnamed: 0,City,ID,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness
45,45,Mataura,6201424,-46.1927,168.8643,78.03,50,4.90,0
143,143,Arang,1278458,21.2000,81.9667,77.45,20,2.33,0
179,179,Makkah al Mukarramah,104515,21.4267,39.8261,71.87,56,4.09,0
239,239,Salalah,286621,17.0151,54.0924,77.00,16,2.30,0
258,258,Manohar Thāna,1263648,24.2400,76.8018,76.60,19,2.17,0
286,286,Māngrol,1263752,21.1167,70.1167,78.33,32,5.64,0
414,414,Flinders,6255012,-34.5833,150.8552,73.00,47,8.01,0
540,540,Haimen,1809062,23.1935,116.6142,71.60,46,2.24,0
551,551,Dhamtari,1272997,20.7072,81.5497,79.29,19,4.41,0
596,596,Kiama,2161515,-34.6833,150.8667,73.00,47,8.01,0


In [4]:
hotels_df=results_df_filtered[["City", "Latitude", "Longitude"]].copy()
hotels_df["Hotel Name"]= ""
hotels_df

,City,Latitude,Longitude,Hotel Name
45,Mataura,-46.1927,168.8643,
143,Arang,21.2000,81.9667,
179,Makkah al Mukarramah,21.4267,39.8261,
239,Salalah,17.0151,54.0924,
258,Manohar Thāna,24.2400,76.8018,
286,Māngrol,21.1167,70.1167,
414,Flinders,-34.5833,150.8552,
540,Haimen,23.1935,116.6142,
551,Dhamtari,20.7072,81.5497,
596,Kiama,-34.6833,150.8667,


In [5]:
os.chdir(r"C:\Users\Yanwho\OneDrive\Documents\python-api-challenge")

In [6]:
from config import google_key

In [7]:
# understaning index,row within a loop - 
# this is needed here because we need to grab the first hotel return from the google api and populate
# the hotel_df and indexing that dataframe will be easier with "index" within the loop
for index,row in hotels_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    name_address=requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json?", params={
            "location": f"{lat},{lng}",
            "radius": 5000,
            "types": "lodging",
            "key": google_key
            })
    name_address=name_address.json()
    try:
        
        hotels_df.loc[index,"Hotel Name"]=name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("missing field - skipping")
    

In [8]:
hotels_df

,City,Latitude,Longitude,Hotel Name
45,Mataura,-46.1927,168.8643,Ellie's Villa
143,Arang,21.2000,81.9667,Hotel Kunwar Shree
179,Makkah al Mukarramah,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
239,Salalah,17.0151,54.0924,HAMDAN PLAZA HOTEL
258,Manohar Thāna,24.2400,76.8018,GEETA SADAN
286,Māngrol,21.1167,70.1167,Hotel The Royal Honours
414,Flinders,-34.5833,150.8552,Shellharbour Resort & Conference Centre
540,Haimen,23.1935,116.6142,Jinghao Business Hotel
551,Dhamtari,20.7072,81.5497,shimon inn
596,Kiama,-34.6833,150.8667,The Sebel Kiama Harbourside


In [9]:
hotels_df.to_csv("VacationPy/hotels.csv")